## Creating POSCAR Files from DCNN-Detected STEM Coordinates and sending to mat3ra servers

This notebook demonstrates how to convert DCNN-predicted atomic coordinates into a POSCAR format, which can be used for DFT simulations using ASE or VASP.


data credits:
    - [Ondej Dyck - ORNL](https://scholar.google.com/citations?user=ZW0OJugAAAAJ&hl=en)



### Assumptions
- The atomic positions from `lattice_coord` are 2D (x, y).
- All atoms lie on a fixed plane (z = 0 Å by default).
- The system is periodic or semi-periodic in the X-Y plane.
- Cell size can be derived from image dimensions or manually set.
- Output will be compatible with ASE and VASP.



### Steps to Create POSCAR

1. **Choose a Frame**
   - Select a frame from your dataset (e.g., `i = 49`) to visualize and export.

2. **Extract Coordinates and Atom Types**
   - Use `lattice_coord` to extract atomic positions.
   - Separate atoms by type:
     - Silicon (Si) atoms have label `1`
     - Carbon (C) atoms have label `0`

3. **Create an ASE `Atoms` Object**
   - Stack all atoms into one list with element symbols.
   - Set their positions as `(x, y, 0.0)`, assuming a flat slab.
   - Define the cell size using the image resolution and pixel size (default: 1 Å/pixel).
   - Enable periodic boundary conditions in the X and Y directions.

4. **Export to POSCAR Format**
   - Use `ase.io.write("filename.POSCAR", atoms, format="vasp")` to save.
   - This format can now be used in DFT software like VASP or further ASE workflows.



In [ ]:
# !gdown https://drive.google.com/uc?id=1-JZSRjIjNjkR0ZQ8ffRDAZ2FID53Yhon
# !gdown https://drive.google.com/uc?id=1-84vLdGFsimD1jaTcGcMzNRCSvjId7-Y
# !gdown https://drive.google.com/uc?id=1-Lowglj7fwEFaJoC9EBKDyfCIsMgOnyu

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
STEM_real = np.load("3DStack13-1-exp.npy")  # raw STEM Image
decoded_imgs = np.load(
    "3DStack13-1-dec.npy"
)  # oytput of DCNN where the each pixel is classified as one of the three classes (C, Si, or background)
lattice_coord = np.load("3DStack13-1-coord.npy", allow_pickle=True)[
    ()
]  # The atomic coodinates found by DCNN

In [ ]:
# Visulaizing one frame of the data, we have 50 frames (0-49)

i = 49  # Choose movie frame

# Squeeze the channels in the predicted image (this is optional)
d_img = np.uint8(decoded_imgs[i] * 255)
d_img = cv2.cvtColor(d_img, cv2.COLOR_BGR2GRAY)
# Get coordinates for C and Si atoms
lattice_coord_ = lattice_coord[i]
coord_Si = lattice_coord[i][np.where(lattice_coord[i][:, 2] == 1)][
    :, 0:2
]  #     silicon atom
coord_C = lattice_coord[i][np.where(lattice_coord[i][:, 2] == 0)][:, 0:2]  # carbon atom
# Plotting
fig = plt.figure(figsize=(15, 10), dpi=100)
ax1 = fig.add_subplot(131)
ax1.imshow(STEM_real[i, :, :, 0], vmin=0, vmax=0.3, cmap="gray")
ax1.axis("off")
ax1.set_title("Experimental", fontsize=14)
ax2 = fig.add_subplot(132)
ax2.imshow(d_img, cmap="jet", interpolation="Gaussian")
ax2.axis("off")
ax2.set_title("DCNN output", fontsize=14)
ax3 = fig.add_subplot(133)
ax3.scatter(coord_Si[:, 1], coord_Si[:, 0], c="red", s=1)
ax3.scatter(coord_C[:, 1], coord_C[:, 0], c="blue", s=1)
ax3.imshow(STEM_real[i, :, :, 0], cmap="gray")
ax3.axis("off")
ax3.set_title("Atomic coordinates", fontsize=14)

In [ ]:
import numpy as np
from ase import Atoms
from ase.io import write
import os

In [ ]:
# Output directory
os.makedirs("poscars", exist_ok=True)

# Assumed calibration: 1 pixel = 1 Å (adjust if needed)
pixel_size = 1.0
num_frames, nx, ny, _ = STEM_real.shape

for i in range(num_frames):
    atoms_all = []
    positions = []

    lattice_coord_ = lattice_coord[i]
    coord_Si = lattice_coord_[np.where(lattice_coord_[:, 2] == 1)][:, :2]
    coord_C = lattice_coord_[np.where(lattice_coord_[:, 2] == 0)][:, :2]

    for coord in coord_C:
        atoms_all.append("C")
        positions.append([coord[1], coord[0], 0.0])

    for coord in coord_Si:
        atoms_all.append("Si")
        positions.append([coord[1], coord[0], 0.0])

    atoms = Atoms(symbols=atoms_all, positions=positions)
    atoms.set_cell([[ny * pixel_size, 0, 0], [0, nx * pixel_size, 0], [0, 0, 10.0]])
    atoms.set_pbc([True, True, False])

    fname = f"poscars/C_Si_{i:02d}.POSCAR"
    write(fname, atoms, format="vasp")

## Lets visualize all the poscars
    - C atom are in grey
    - The Si atoms are in red

![graphene with Si dopants](../../assests/C_Si.gif)

## validating the POSCAR

In [ ]:
from ase.io import read
import os

folder = "poscars"
files = sorted([f for f in os.listdir(folder) if f.endswith(".POSCAR")])

for fname in files:
    path = os.path.join(folder, fname)
    try:
        atoms = read(path, format="vasp")
        print(f"{fname}: {len(atoms)} atoms, cell={atoms.cell.lengths()}")
    except Exception as e:
        print(f"ERROR in {fname}: {e}")

In [ ]:
from ase.visualize import view

atoms = read("poscars/C_Si_00.POSCAR", format="vasp")
view(atoms)  # launches ASE GUI

In [ ]:
atoms = read("poscars/C_Si_00.POSCAR")
print("Chemical symbols:", atoms.get_chemical_symbols())
print("Cell lengths:", atoms.cell.lengths())
print("Is PBC:", atoms.get_pbc())

## Sending to Mat3RA servers

In [ ]:
import yaml

# Load the YAML file
with open("credential.yaml", "r") as f:
    cred = yaml.safe_load(f)

ACCOUNT_ID = cred["accountId"]
AUTH_TOKEN = cred["authToken"]  # @param {type:"string"}
# # MATERIALS_PROJECT_API_KEY = cred["materials-project-apikey"] # @param {type:"string"}
ORGANIZATION_ID = ACCOUNT_ID
OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID

In [ ]:
import sys
import os

sys.path.append("/Users/utkarshpratiush/project/api-examples/")

In [ ]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import display_JSON

from exabyte_api_client.endpoints.materials import MaterialEndpoints

In [ ]:
ENDPOINT_ARGS = cred["endpointArgs"]

In [ ]:
endpoint = MaterialEndpoints(*ENDPOINT_ARGS)

In [ ]:
import os

# Directory containing POSCAR files
POSCAR_DIR = "poscars"

# Get all POSCAR files in the directory
poscar_files = [f for f in os.listdir(POSCAR_DIR) if f.endswith(".POSCAR")]
print(f"Found {len(poscar_files)} POSCAR files in {POSCAR_DIR}")

In [ ]:
def upload_poscar(filepath, name):
    """Upload a single POSCAR file to Mat3ra server"""
    try:
        with open(filepath) as f:
            content = f.read()
        material = endpoint.import_from_file(name, content, owner_id=OWNER_ID)
        return material
    except Exception as e:
        print(f"Error uploading {filepath}: {str(e)}")
        return None

In [ ]:
# Process all POSCAR files
uploaded_materials = []

for poscar_file in poscar_files:
    print(f"Processing {poscar_file}...")
    # Extract name from filename (remove .POSCAR extension)
    name = os.path.splitext(poscar_file)[0]
    filepath = os.path.join(POSCAR_DIR, poscar_file)

    material = upload_poscar(filepath, name)
    if material:
        uploaded_materials.append(material)
        print(f"Successfully uploaded {name}")

print(f"\nUploaded {len(uploaded_materials)} materials to Mat3ra server")